In [2]:
suppressMessages(library(rwwa))

# Fit linear model to gridded data

## Edit this cell to load the correct data and define the model you want to fit

In [6]:
# use the same filename from the last workbook
fnm_root = "tx30x-era5"

# set the event year and warming level you're interested in
ev_year <- 2024
gmst_change <- 1.3

# specify the statistical model you want to fit (should be the same as for your main event definition)
dist <- "gev"
fittype <- "shift"
cov_nm <- "gmst"
lower = F            # use T to look at high extremes, F to look at low extremes

## Run these cells with no changes to fit the model at each grid cell

In [7]:
# load covariate data
gmst = read.table("gmst.txt", col.names = c("year", "gmst"))
gmst$gmst <- gmst$gmst - gmst$gmst[gmst$year == ev_year]

# define factual & counterfactual covariates
cov_f <- gmst[gmst$year == ev_year,"gmst", drop = F]
cov_cf <- rbind("pi" = cov_f - gmst_change)

In [8]:
# loop over all flattened data & fit nonstationary model to each - NB no bootstrapping
fl = list.files("flattened", full.names = T)
for(fnm in fl) {
    
    res_fnm <- gsub("flattened", "flattened-res", fnm)
    
    if (!file.exists(res_fnm)) {
        
        # load the .csv and 
        df <- read.csv(fnm, skip = 3, header = F)
        colnames(df)[1] <- "year"
        df <- merge(gmst, df)

        # fit nonstationary model to all columns
        models <- sapply(colnames(df)[grepl("V", colnames(df))], function(cnm) {
            df_c <- df[,c("year", "gmst", cnm)]
            tryCatch({
                fit_ns(dist, fittype, df_c, cnm, cov_nm, lower = lower, ev_year = ev_year)
            }, error = function(cond) { return(NULL) })
        }, simplify = F)

        # get all model results & save as .csv to be reconstructed & plotted in python
        fitted_res <- sapply(models, function(mdl) {
            if(!is.null(unlist(mdl)[1])) {
                mdl_ests(mdl, cov_f = cov_f, cov_cf = cov_cf)
            } else {
                rep(NA, 11)
            }})
        write.csv(fitted_res, res_fnm)
    }
}

**Now open the next workbook (using python) to reconstruct the fitted results into a map**